In [32]:
import pandas as pd # type: ignore
import statsmodels.api as sm # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.linear_model import LinearRegression # type: ignore
from sklearn.metrics import mean_squared_error, r2_score # type: ignore
from sklearn.ensemble import RandomForestRegressor # type: ignore
import numpy as np # type: ignore
from sklearn.decomposition import PCA # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore
import matplotlib.pyplot as plt # type: ignore

In [5]:
def get_df(file_path, sheet_name):
    return pd.read_excel(file_path, sheet_name)

program1_train_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TrainingData.xlsx", "Program1")
program2_train_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TrainingData.xlsx", "Program2")
program3_train_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TrainingData.xlsx", "Program3")
program4_train_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TrainingData.xlsx", "Program4")

program1_test_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TestData.xlsx", "Program1")
program2_test_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TestData.xlsx", "Program2")
program3_test_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TestData.xlsx", "Program3")
program4_test_df = get_df("Z:/BSA Data/DataSplit/BSA-DataSet_2122_2223-TestData.xlsx", "Program4")

print(program1_train_df.columns)

Index(['train', 'Gender', 'Nationality', 'PreEducation', 'Program', 'Year',
       'BSA', 'Credits-Y1', 'Course3-1', 'Course9-1', 'Course8-1', 'Course7-1',
       'Course3-R', 'Course9-R', 'Crd-B1B2', 'Course23', 'Course26', 'Course3',
       'Course16', 'Course8', 'Course9', 'Course22', 'Course7', 'Course25',
       'Course24', 'Course23-R', 'Course26-R', 'Course16-R', 'Course8-R',
       'Course22-R', 'Course7-R', 'Course25-R', 'Course24-R'],
      dtype='object')


In [6]:
# Function to perform the analysis
def program_analysis(train_df, test_df, program_name):
    # Combine dfs
    combined_df = pd.concat([train_df, test_df], keys=['train', 'test'])

    # Get dummies and add personal extra variables
    b1_index = [8,9,12,13]
    b2_index = [10,11]
    combined_df['Crd-B1'] = combined_df.iloc[:, b1_index].sum(axis=1)
    combined_df['Crd-B2'] = combined_df.iloc[:, b2_index].sum(axis=1)
    dummifiable_columns = ['Gender', 'Nationality', 'PreEducation','Year']
    dummies = pd.get_dummies(combined_df[dummifiable_columns], dtype=int)
    combined_df = combined_df.drop(columns=dummifiable_columns)
    combined_df = pd.concat([combined_df, dummies], axis = 1)

    # Split back into original dfs
    train_df = combined_df.xs('train')
    test_df = combined_df.xs('test')

    # Exclude the unnecessary columns
    train_df = train_df.drop(columns=['train', 'Program'])
    test_df = test_df.drop(columns=['train', 'Program'])
    
    # Identify numeric columns only
    numeric_cols = train_df.select_dtypes(include=[float, int]).columns.tolist()
    numeric_cols_2 = test_df.select_dtypes(include=[float, int]).columns.tolist()

    # Prepare the data
    X_train = train_df[numeric_cols].drop(columns=['Credits-Y1'])
    y_train = train_df['Credits-Y1']
    X_test = test_df[numeric_cols_2].drop(columns=['Credits-Y1'])
    y_test = test_df['Credits-Y1']

    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Correlation Analysis
    corr_matrix = train_df[numeric_cols].corr()
    corr_with_credits = corr_matrix["Credits-Y1"].sort_values(ascending=False)
    print(f"Correlation with Credits-Y1 ({program_name} Training Data):")
    print(corr_with_credits)

    # Regression Analysis with statsmodels
    X_train_sm = sm.add_constant(X_train)
    model_sm = sm.OLS(y_train, X_train_sm).fit()
    print(f"\nRegression Analysis ({program_name} statsmodels):")
    print(model_sm.summary())

In [7]:
programs = [
    ("Program 1", program1_train_df, program1_test_df),
    ("Program 2", program2_train_df, program2_test_df),
    ("Program 3", program3_train_df, program3_test_df),
    ("Program 4", program4_train_df, program4_test_df)
]

for program_name, train_df, test_df in programs:
    program_analysis(train_df, test_df, program_name)

Correlation with Credits-Y1 (Program 1 Training Data):
Credits-Y1                             1.000000
Crd-B1B2                               0.872440
Crd-B2                                 0.821399
Course8                                0.819256
Course22                               0.797311
Course25                               0.770375
Course8-1                              0.756926
Course23                               0.751058
Course9                                0.729035
Course22-R                             0.711987
Course24                               0.706904
Course7                                0.705486
Course9-1                              0.685360
Course25-R                             0.678141
Crd-B1                                 0.676326
Course8-R                              0.661915
Course3-R                              0.657432
Course7-R                              0.635603
Course23-R                             0.620445
Course3                          

In [70]:
def data_prep(train_df, test_df, program_name):
    # Combine all training and testing dataframes
    combined_df = pd.concat([train_df, test_df], keys=['training', 'testing'])

    # Create dummies
    dummifiable_columns = ['Gender', 'Nationality', 'PreEducation', 'Program', 'Year', 'BSA']
    dummies = pd.get_dummies(combined_df[dummifiable_columns], dtype=int)
    combined_df = combined_df.drop(columns=dummifiable_columns)
    combined_df = pd.concat([combined_df.iloc[:, :1], dummies, combined_df.iloc[:, 1:]], axis = 1)

    # Split back into original dataframes
    train_df = combined_df.xs('training')
    test_df = combined_df.xs('testing')

    # Exclude the unnecessary columns
    train_df = train_df.drop(columns=['train'])
    test_df = test_df.drop(columns=['train'])

    # Select relevant columns (up to "Crd-B1B2")
    columns_to_include = train_df.columns[:train_df.columns.tolist().index("Crd-B1B2") + 1]
    train_df = train_df[columns_to_include]
    test_df = test_df[columns_to_include]

    return train_df, test_df

# Analysis on relevant columns only
def relevant_analysis(train_df, test_df, program_name):
    # Prepare Data for Analysis
    train_df, test_df = data_prep(train_df, test_df, program_name)

    # Separate features and target variable
    X_train = train_df.drop(columns=['Credits-Y1'])
    y_train = train_df['Credits-Y1']
    X_test = test_df.drop(columns=['Credits-Y1'])
    y_test = test_df['Credits-Y1']

    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Correlation Analysis
    corr_matrix = train_df.corr()
    corr_with_credits = corr_matrix["Credits-Y1"].sort_values(ascending=False)
    print(f"Correlation with Credits-Y1 ({program_name} Training Data):")
    print(corr_with_credits)

    # Regression Analysis with statsmodels
    X_train_sm = sm.add_constant(X_train)
    model_sm = sm.OLS(y_train, X_train_sm).fit()
    print(f"\nRegression Analysis ({program_name} statsmodels):")
    print(model_sm.summary())

In [71]:
for program_name, train_df, test_df in programs:
    relevant_analysis(train_df, test_df, program_name)

Correlation with Credits-Y1 (Program 1 Training Data):
Credits-Y1                             1.000000
BSA_PS                                 0.886314
Crd-B1B2                               0.872440
Course8-1                              0.756926
Course9-1                              0.685360
Course3-R                              0.657432
Course9-R                              0.602428
Course7-1                              0.599292
Course3-1                              0.566055
PreEducation_Buitenlands               0.208107
Nationality_Azie                       0.151701
Nationality_EU                         0.133323
Gender_F                               0.115673
Nationality_Europa                     0.066351
Nationality_Oceanie                    0.046360
Year_22/23                             0.035822
Nationality_Onbekend                   0.032768
Nationality_Mid-Zuid-Amerika           0.027756
Nationality_Afrika                     0.011906
BSA_DI                           

In [81]:
def perform_pca_analysis(train_df, test_df, program_name):
    # Prepare Data for Analysis
    train_df2, test_df2 = data_prep(train_df, test_df, program_name)

    # Select relevant columns (up to "Crd-B1B2")
    columns_to_include = train_df2.columns[:train_df2.columns.tolist().index("Crd-B1B2") + 1]
    train_df2 = train_df2[columns_to_include]
    test_df2 = test_df2[columns_to_include]

    # Separate features and target variable
    X_train = train_df2.drop(columns=['Credits-Y1'])
    y_train = train_df2['Credits-Y1']
    X_test = test_df2.drop(columns=['Credits-Y1'])
    y_test = test_df2['Credits-Y1']

    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)
    y_train = y_train.replace([np.inf, -np.inf], np.nan).fillna(0)
    y_test = y_test.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Standardize the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply PCA
    pca = PCA()
    pca.fit(X_train_scaled)

    # Explained variance ratio
    explained_variance = pca.explained_variance_ratio_
    print(f"Explained Variance Ratio for {program_name}:")
    print(explained_variance)

    # Determine the number of components to retain (e.g., explain at least 95% variance)
    total_variance = 0
    num_components = 0
    for variance in explained_variance:
        total_variance += variance
        num_components += 1
        if total_variance >= 0.95:
            break

    print(f"Number of components to retain for {program_name}: {num_components}\n")

    # Transform data using the selected number of components
    X_train_pca = pca.transform(X_train_scaled)[:, :num_components]
    X_test_pca = pca.transform(X_test_scaled)[:, :num_components]

    # Create DataFrames with the principal components
    pca_columns = [f'PC{i+1}' for i in range(num_components)]
    X_train_pca_df = pd.DataFrame(X_train_pca, columns=pca_columns)
    X_test_pca_df = pd.DataFrame(X_test_pca, columns=pca_columns)

    # Ensure that y_train and y_test are series
    if isinstance(y_train, pd.DataFrame):
        y_train = y_train.iloc[:, 0]
    if isinstance(y_test, pd.DataFrame):
        y_test = y_test.iloc[:, 0]

    # Add the target variable 'Credits-Y1' back to the DataFrames
    X_train_pca_df['Credits-Y1'] = y_train.reset_index(drop=True)
    X_test_pca_df['Credits-Y1'] = y_test.reset_index(drop=True)

    return X_train_pca_df, X_test_pca_df

In [82]:
pca_results = {}

for program_name, train_df, test_df in programs:
    X_train_pca_df, X_test_pca_df = perform_pca_analysis(train_df, test_df, program_name)
    pca_results[program_name] = {
        'train': X_train_pca_df,
        'test': X_test_pca_df
    }

# Now you can access each program's PCA-transformed dataframes from the `pca_results` dictionary
# Example: Accessing Program 1's transformed data
program1_train_pca_df = pca_results['Program 1']['train']
program1_test_pca_df = pca_results['Program 1']['test']

Explained Variance Ratio for Program 1:
[1.70492634e-01 9.66144055e-02 7.03293858e-02 6.32424429e-02
 4.47784303e-02 4.37253137e-02 3.89545980e-02 3.51776698e-02
 3.41372834e-02 3.26402726e-02 3.16488713e-02 3.15527393e-02
 3.14788846e-02 3.14099796e-02 3.12703744e-02 3.10475253e-02
 3.05441257e-02 3.03624261e-02 2.68560765e-02 2.40696545e-02
 1.88140036e-02 1.35209785e-02 1.24781164e-02 1.01017393e-02
 6.37330808e-03 5.72237049e-03 2.65639080e-03 7.95800811e-17
 2.49243352e-17 8.46231074e-19 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
Number of components to retain for Program 1: 22

Explained Variance Ratio for Program 2:
[2.00846517e-01 1.20603311e-01 8.78995060e-02 7.11545986e-02
 5.76786671e-02 4.92681719e-02 4.32988826e-02 4.05818882e-02
 3.86765922e-02 3.68222744e-02 3.61768429e-02 3.54871404e-02
 3.51832265e-02 3.29089582e-02 2.68770303e-02 2.36954418e-02
 2.05430570e-02 1.17790060e-02 8.79584280e-03 6.89398970e-03
 5.91694341e-03 5.55991691e-03 3.35219544e-03 1.79008265e-16

In [83]:
for program_name in pca_results.keys():
    X_train = pca_results[program_name]["train"].drop(columns = ["Credits-Y1"])
    y_train = pca_results[program_name]["train"]["Credits-Y1"]
    X_test = pca_results[program_name]["test"].drop(columns = ["Credits-Y1"])
    y_test = pca_results[program_name]["test"]["Credits-Y1"]

    # Initialize and train the dataset
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # Make predictions
    y_pred = lr_model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"{program_name}")
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")
    print(f"\n")

Program 1
Mean Squared Error: 43.796055232931
R2 Score: 0.9168578783512282


Program 2
Mean Squared Error: 65.59138987224861
R2 Score: 0.898907518642531


Program 3
Mean Squared Error: 40.84919461495996
R2 Score: 0.8990882902123214


Program 4
Mean Squared Error: 34.760924832834654
R2 Score: 0.8518984072564668


